In [1]:
import math
#import torch
#import gpytorch
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np
from scipy.stats import norm
#from gp_dev.core import *
#from ddop.datasets import load_yaz
from pathlib import Path
import datetime
import category_encoders as ce
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, RationalQuadratic, ExpSineSquared

In [3]:
path = Path("../../..")
path_ds = path/'datasets'
path_res = path/'res_data'
path_plot = path/'plots'

In [4]:
df = pd.read_csv(path_ds/'lh_data.csv').drop(columns='Unnamed: 0')

In [5]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [6]:
df.head()

,BS_TS_INBOUND,HOUR0,HOUR1,HOUR2,HOUR3,HOUR4,HOUR5,HOUR6,HOUR7,HOUR8,HOUR9,HOUR10,HOUR11,HOUR12,HOUR13,HOUR14,HOUR15,HOUR16,HOUR17,HOUR18,HOUR19,Avis_Warenan_Z1_Z2,Weekday_Mean_Avis,perc_diff_avis_weekday,Called_Avis_Warenan_Z1_Z2,Weekday_Mean_Called_Avis,perc_diff_called_avis_weekday,DE_1,DE_100,DE_2,DE_3,DE_4,DE_5,DE_6,DE_7,FAST_1,FAST_100,FAST_2,FAST_3,FAST_4,FAST_5,FAST_6,FAST_7,REST_1,REST_100,REST_2,REST_3,REST_4,REST_5,REST_6,REST_7,US_1,US_100,US_2,US_3,US_4,US_5,US_6,US_7,US_Total_On_the_Road,DE_Total_On_the_Road,FAST_Total_On_the_Road,REST_Total_On_the_Road,bhday1_USA,bhday2_USA,bhday3_USA,ahday1_USA,ahday2_USA,ahday3_USA,bhday1_D,bhday2_D,bhday3_D,ahday1_D,ahday2_D,ahday3_D,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,total,index.num,year,half,quarter,month,day,wday.xts,qday,yday,mweek,week,week.iso,week2,week3,week4,mday7,isMonday,isTuesday,isWednesday,isThursday,isFriday,isSaturday,LagDemandA_1Day,LagDemandA_2Day,LagDemandA_3Day,LagDemandA_4Day,LagDemandA_5Day,LagDemandA_6Day,LagDemandA_7Day,LagDemandA_14Day,LagDemandA_21Day,LagDemandA_28Day,LagDemandB_1Day,LagDemandB_2Day,LagDemandB_3Day,LagDemandB_4Day,LagDemandB_5Day,LagDemandB_6Day,LagDemandB_7Day,LagDemandB_14Day,LagDemandB_21Day,LagDemandB_28Day,LagDemandA_Sum3Day,LagDemandA_Sum7Day,LagDemandB_Sum3Day,LagDemandB_Sum7Day,LagDemand_1Day,LagDemand_2Day,LagDemand_3Day,LagDemand_4Day,LagDemand_5Day,LagDemand_6Day,LagDemand_7Day,LagDemand_14Day,LagDemand_21Day,LagDemand_28Day,LagDemand_Sum3Day,LagDemand_Sum7Day
0,2016-02-09,0,0,0,0,0,0,167,201,57,40,241,119,76,108,101,93,44,67,16,0,0.353973,0.508114,0.366845,0.233556,0.642752,0.065362,0.257143,0.083333,0.051724,0.037037,0.032258,0.037037,0.000000,0.000000,0.037838,0.083333,0.000000,0.000000,0.042553,0.000000,0.000000,0.111111,0.172414,0.076923,0.000000,0.000000,0.030303,0.466667,0.022727,0.000,0.100865,0.066524,0.000000,0.003650,0.013158,0.022222,0.017937,0.004484,0.092246,0.308411,0.037879,0.260870,0,0,0,0,0,0,0,0,0,0,0,0,0.600923,0.039757,0.080488,0.415515,0.372596,0.398361,0.336163,0.426786,0.005410,0.008574,0.378430,0.316812,0.450100,0.383090,0.460401,0.009585,0.002488,0.528749,0.204082,0.310827,0.478107,0.326506,0.001206,0.009589,0.356181,0.233702,0.251852,0.244085,0.359494,0.582769,0.000000,0,0,0.0,0.090909,0.266667,0.2,0.428571,0.104396,0.4,0.096154,0.098039,0,0.0,0.666667,0.25,0,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,2016-02-10,55,0,0,0,0,0,27,64,35,178,41,143,124,176,156,62,61,4,20,34,0.248710,0.596723,0.224781,0.313632,0.677617,0.103387,0.000000,0.083333,0.413793,0.000000,0.032258,0.000000,0.000000,0.000000,0.081081,0.083333,0.008197,0.000000,0.000000,0.045455,0.000000,0.000000,0.057471,0.076923,0.111111,0.000000,0.000000,0.000000,0.000000,0.025,0.162824,0.064378,0.047231,0.000000,0.000000,0.004444,0.013453,0.013453,0.179144,0.233645,0.064394,0.130435,0,0,0,0,0,0,0,0,0,0,0,0,0.588391,0.535040,0.029268,0.069892,0.395833,0.356557,0.387097,0.322321,0.417493,0.003215,0.005676,0.370262,0.299401,0.443633,0.357973,0.454739,0.011194,0.002255,0.510204,0.194412,0.306509,0.466265,0.314837,0.001370,0.006120,0.340713,0.232099,0.251557,0.245570,0.620923,0.001577,0,0,0.0,0.090909,0.300000,0.4,0.439560,0.107143,0.4,0.096154,0.098039,0,0.0,0.666667,0.25,0,0,1,0,0,0,0.595632,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.329569,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.210515,0.123244,0.148542,0.105696,0.654528,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.246023,0.139662
2,2016-02-11,309,1,1,0,0,0,59,110,51,178,138,266,171,137,50,36,40,57,2,0,0.312693,0.694059,0.262339,0.493804,0.684639,0.166555,0.114286,0.083333,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.227027,0.000000,0.016393,0.000000,0.000000,0.000000,0.111111,0.000

In [7]:
df.columns

Index(['BS_TS_INBOUND', 'HOUR0', 'HOUR1', 'HOUR2', 'HOUR3', 'HOUR4', 'HOUR5',
       'HOUR6', 'HOUR7', 'HOUR8',
       ...
       'LagDemand_3Day', 'LagDemand_4Day', 'LagDemand_5Day', 'LagDemand_6Day',
       'LagDemand_7Day', 'LagDemand_14Day', 'LagDemand_21Day',
       'LagDemand_28Day', 'LagDemand_Sum3Day', 'LagDemand_Sum7Day'],
      dtype='object', length=163)

In [90]:
# removing additional week and month related categorical features
df_sel = df.loc[:, ~(df.columns.isin(['BS_TS_INBOUND','day','wday.xts', 'qday','yday','mweek','week', 'week.iso','week2','week3','week4','mday7']))]

# one hot encode categorical variables

for cat in ['year', 'half', 'quarter', 'month']:
    encoder = ce.OneHotEncoder()
    df_sel = pd.concat([df_sel, encoder.fit_transform(df_sel[cat].astype('category'))], axis=1).drop(columns = cat)
    #test_x_df = pd.concat([test_x_df, encoder.transform(test_x_df[cat].astype('category'))], axis=1).drop(columns = cat)

/media/data/anaconda3/envs/gp_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/media/data/anaconda3/envs/gp_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/media/data/anaconda3/envs/gp_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/media/data/anaconda3/envs/gp_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_cat

In [91]:
df_sel.head()

,HOUR0,HOUR1,HOUR2,HOUR3,HOUR4,HOUR5,HOUR6,HOUR7,HOUR8,HOUR9,HOUR10,HOUR11,HOUR12,HOUR13,HOUR14,HOUR15,HOUR16,HOUR17,HOUR18,HOUR19,Avis_Warenan_Z1_Z2,Weekday_Mean_Avis,perc_diff_avis_weekday,Called_Avis_Warenan_Z1_Z2,Weekday_Mean_Called_Avis,perc_diff_called_avis_weekday,DE_1,DE_100,DE_2,DE_3,DE_4,DE_5,DE_6,DE_7,FAST_1,FAST_100,FAST_2,FAST_3,FAST_4,FAST_5,FAST_6,FAST_7,REST_1,REST_100,REST_2,REST_3,REST_4,REST_5,REST_6,REST_7,US_1,US_100,US_2,US_3,US_4,US_5,US_6,US_7,US_Total_On_the_Road,DE_Total_On_the_Road,FAST_Total_On_the_Road,REST_Total_On_the_Road,bhday1_USA,bhday2_USA,bhday3_USA,ahday1_USA,ahday2_USA,ahday3_USA,bhday1_D,bhday2_D,bhday3_D,ahday1_D,ahday2_D,ahday3_D,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,total,index.num,isMonday,isTuesday,isWednesday,isThursday,isFriday,isSaturday,LagDemandA_1Day,LagDemandA_2Day,LagDemandA_3Day,LagDemandA_4Day,LagDemandA_5Day,LagDemandA_6Day,LagDemandA_7Day,LagDemandA_14Day,LagDemandA_21Day,LagDemandA_28Day,LagDemandB_1Day,LagDemandB_2Day,LagDemandB_3Day,LagDemandB_4Day,LagDemandB_5Day,LagDemandB_6Day,LagDemandB_7Day,LagDemandB_14Day,LagDemandB_21Day,LagDemandB_28Day,LagDemandA_Sum3Day,LagDemandA_Sum7Day,LagDemandB_Sum3Day,LagDemandB_Sum7Day,LagDemand_1Day,LagDemand_2Day,LagDemand_3Day,LagDemand_4Day,LagDemand_5Day,LagDemand_6Day,LagDemand_7Day,LagDemand_14Day,LagDemand_21Day,LagDemand_28Day,LagDemand_Sum3Day,LagDemand_Sum7Day,year_1,year_2,half_1,half_2,quarter_1,quarter_2,quarter_3,quarter_4,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,0,0,0,0,0,0,167,201,57,40,241,119,76,108,101,93,44,67,16,0,0.353973,0.508114,0.366845,0.233556,0.642752,0.065362,0.257143,0.083333,0.051724,0.037037,0.032258,0.037037,0.000000,0.000000,0.037838,0.083333,0.000000,0.000000,0.042553,0.000000,0.000000,0.111111,0.172414,0.076923,0.000000,0.000000,0.030303,0.466667,0.022727,0.000,0.100865,0.066524,0.000000,0.003650,0.013158,0.022222,0.017937,0.004484,0.092246,0.308411,0.037879,0.260870,0,0,0,0,0,0,0,0,0,0,0,0,0.600923,0.039757,0.080488,0.415515,0.372596,0.398361,0.336163,0.426786,0.005410,0.008574,0.378430,0.316812,0.450100,0.383090,0.460401,0.009585,0.002488,0.528749,0.204082,0.310827,0.478107,0.326506,0.001206,0.009589,0.356181,0.233702,0.251852,0.244085,0.359494,0.582769,0.000000,0,1,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,55,0,0,0,0,0,27,64,35,178,41,143,124,176,156,62,61,4,20,34,0.248710,0.596723,0.224781,0.313632,0.677617,0.103387,0.000000,0.083333,0.413793,0.000000,0.032258,0.000000,0.000000,0.000000,0.081081,0.083333,0.008197,0.000000,0.000000,0.045455,0.000000,0.000000,0.057471,0.076923,0.111111,0.000000,0.000000,0.000000,0.000000,0.025,0.162824,0.064378,0.047231,0.000000,0.000000,0.004444,0.013453,0.013453,0.179144,0.233645,0.064394,0.130435,0,0,0,0,0,0,0,0,0,0,0,0,0.588391,0.535040,0.029268,0.069892,0.395833,0.356557,0.387097,0.322321,0.417493,0.003215,0.005676,0.370262,0.299401,0.443633,0.357973,0.454739,0.011194,0.002255,0.510204,0.194412,0.306509,0.466265,0.314837,0.001370,0.006120,0.340713,0.232099,0.251557,0.245570,0.620923,0.001577,0,0,1,0,0,0,0.595632,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.329569,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.210515,0.123244,0.148542,0.105696,0.654528,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.246023,0.139662,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,309,1,1,0,0,0,59,110,51,178,138,266,171,137,50,36,40,57,2,0,0.312693,0.694059,0.262339,0.493804,0.684639,0.166555,0.114286,0.083333,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.227027,0.000000,0.016393,0.000000,0.000000,0.000000,0.111111,0.000000,0.310345,0.025641,0.055556,0.042553,0.000000,0.000000,0.000000,0.00

In [116]:
res = pd.DataFrame()
for hour in range(20):
    
    # make train/val/test
    n_train = 425
    train_x, train_y = df_sel.iloc[:n_train, 20:], df_sel.loc[:n_train-1, 'HOUR'+str(hour)]
    test_x, test_y = df_sel.iloc[n_train:, 20:], df_sel.loc[n_train-1:, 'HOUR'+str(hour)]
    
    print(train_x.shape)
    print(test_x.shape)

    train_y = train_y.values
    test_y = test_y.values

    # Kernel with parameters given in GPML book
    k1 = 1**2 * RBF(length_scale=0.261)  # long term smooth rising trend
    k2 = 2.4**2 * RBF(length_scale=90.0) \
        * ExpSineSquared(length_scale=1.3, periodicity=1.0)  # seasonal component
    k3 = 0.66**2 \
        * RationalQuadratic(length_scale=1.2, alpha=0.78) # medium term irregularity
    k4 = 0.18**2 * RBF(length_scale=0.134) \
        + WhiteKernel(noise_level=1.09**2)  # noise terms

    kernel_gpml = k1 + k4

    gp = GaussianProcessRegressor(kernel=kernel_gpml, normalize_y=True, n_restarts_optimizer=10)#, alpha=1)
    gp.fit(train_x, train_y)

    print("\nLearned kernel: %s" % gp.kernel_)
    print("Log-marginal-likelihood: %.3f"
          % gp.log_marginal_likelihood(gp.kernel_.theta))

    nv_means, y_std = gp.predict(test_x,  return_std=True)
    nv_sigma = y_std

    df_tmp=pd.DataFrame({"hour":np.repeat(hour, nv_means.shape[0]), 'idx_testset': range(n_train, df_sel.shape[0]), 'mean': nv_means, 'sigma': nv_sigma})
    res = pd.concat([res, df_tmp], ignore_index=True)
    
df_res = pd.DataFrame(res)

(425, 147)
(107, 147)

Learned kernel: 0.815**2 * RBF(length_scale=6.57e-05) + 0.682**2 * RBF(length_scale=2.55) + WhiteKernel(noise_level=0.0137)
Log-marginal-likelihood: -569.911
(425, 147)
(107, 147)

Learned kernel: 0.664**2 * RBF(length_scale=0.207) + 0.169**2 * RBF(length_scale=0.134) + WhiteKernel(noise_level=0.531)
Log-marginal-likelihood: -603.049
(425, 147)
(107, 147)

Learned kernel: 0.364**2 * RBF(length_scale=2.33) + 0.936**2 * RBF(length_scale=0.0149) + WhiteKernel(noise_level=0.00769)
Log-marginal-likelihood: -594.619
(425, 147)
(107, 147)

Learned kernel: 0.97**2 * RBF(length_scale=0.000588) + 0.224**2 * RBF(length_scale=2.53) + WhiteKernel(noise_level=0.0187)
Log-marginal-likelihood: -601.382
(425, 147)
(107, 147)

Learned kernel: 1.12**2 * RBF(length_scale=1.17) + 0.00316**2 * RBF(length_scale=1e-05) + WhiteKernel(noise_level=1e-05)
Log-marginal-likelihood: -578.817
(425, 147)
(107, 147)

Learned kernel: 0.454**2 * RBF(length_scale=2.9) + 0.931**2 * RBF(length_scale=0

df_res.to_csv(path_res/'gp_mean_sigma_per_hour_for_testset')